## Salvado o SVR

In [7]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib

def treinar_svr_e_salvar(caminho_csv, coluna, look_back=10):
    dados = pd.read_csv(caminho_csv)[coluna].values
    scaler = MinMaxScaler()
    dados_scaled = scaler.fit_transform(dados.reshape(-1, 1)).flatten()

    X, y = [], []
    for i in range(len(dados_scaled) - look_back):
        X.append(dados_scaled[i:i+look_back])
        y.append(dados_scaled[i+look_back])
    X, y = np.array(X), np.array(y)

    split = int(0.7 * len(X))
    X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

    model = SVR(C=10, epsilon=0.01, kernel='rbf')
    model.fit(X_train, y_train)

    # Salvar modelo e scaler
    joblib.dump(model, f"svr_{coluna}.joblib")
    joblib.dump(scaler, f"scaler_{coluna}.joblib")

    print(f"[SVR] {coluna} modelo salvo como svr_{coluna}.joblib")

In [8]:
treinar_svr_e_salvar("D3_component.csv", "D3")
treinar_svr_e_salvar("D2_component.csv", "D2")
treinar_svr_e_salvar("D1_component.csv", "D1")

[SVR] D3 modelo salvo como svr_D3.joblib
[SVR] D2 modelo salvo como svr_D2.joblib
[SVR] D1 modelo salvo como svr_D1.joblib


## SVR para multiplos dias

In [18]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib

def treinar_svr_multiplos_passos(caminho_csv, coluna, passos=[1, 3, 7, 30]):
    dados = pd.read_csv(caminho_csv)[coluna].values
    dados = dados.reshape(-1)  # já assumimos normalizado externamente

    for passo in passos:
        look_back = 5 if passo in [1, 3, 5] else 10

        print(f"⏳ Treinando SVR para {coluna} | t+{passo} (look_back={look_back})")

        X, y = criar_dataset_multi_step(dados, look_back=look_back, passo=passo)
        split = int(0.7 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = SVR(C=10, epsilon=0.01, kernel='rbf')
        model.fit(X_train, y_train)

        joblib.dump(model, f"svr_{coluna}_t{passo}.joblib")
        print(f"✅ Modelo SVR salvo: svr_{coluna}_t{passo}.joblib")




## SRV com Grid e Pipeline

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error
import joblib

def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i+look_back])
        y.append(series[i+look_back+passo-1])
    return np.array(X), np.array(y)

def treinar_svr_multiplos_passos(caminho_csv, coluna, passos=[1, 3, 5, 7, 30]):
    print(f"\n📈 GridSearchCV para SVR + RobustScaler no componente: {coluna}")

    # 1. Carregar os dados reais da componente (sem normalização)
    dados = pd.read_csv(caminho_csv)[coluna].values.reshape(-1)

    for passo in passos:
        look_back = 5 if passo in [1, 3, 5] else 10
        print(f"\n⏳ Treinando SVR | {coluna} | t+{passo} | look_back={look_back}")

        X, y = criar_dataset_multi_step(dados, look_back=look_back, passo=passo)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # 2. Definir o pipeline
        pipeline = Pipeline([
            ('scaler', RobustScaler()),
            ('svr', SVR())
        ])

        # 3. Grade de hiperparâmetros
        param_grid = {
            'svr__C': [1, 10, 50],
            'svr__epsilon': [0.01, 0.1, 0.2],
            'svr__gamma': ['scale', 'auto']
        }

        # 4. Configurar Grid Search com MAE
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=make_scorer(mean_absolute_error, greater_is_better=False),
            cv=5,
            n_jobs=-1,
            verbose=1
        )

        # 5. Treinar com validação cruzada
        grid_search.fit(X_train, y_train)

        # 6. Melhor modelo encontrado
        melhor_pipeline = grid_search.best_estimator_
        print(f"🔍 Melhor conjunto de parâmetros: {grid_search.best_params_}")

        # 7. Salvar o pipeline ajustado
        joblib.dump(melhor_pipeline, f"svr_{coluna}_t{passo}.joblib")
        print(f"✅ Pipeline final salvo: svr_{coluna}_t{passo}.joblib")

    print("\n✅ Todos os modelos treinados com GridSearch e salvos com sucesso.")


In [2]:
treinar_svr_multiplos_passos("A3_component.csv", "A3")
treinar_svr_multiplos_passos("D2_component.csv", "D2")
treinar_svr_multiplos_passos("D1_component.csv", "D1")



📈 GridSearchCV para SVR + RobustScaler no componente: A3

⏳ Treinando SVR | A3 | t+1 | look_back=5
Fitting 5 folds for each of 18 candidates, totalling 90 fits
🔍 Melhor conjunto de parâmetros: {'svr__C': 10, 'svr__epsilon': 0.2, 'svr__gamma': 'auto'}
✅ Pipeline final salvo: svr_A3_t1.joblib

⏳ Treinando SVR | A3 | t+3 | look_back=5
Fitting 5 folds for each of 18 candidates, totalling 90 fits
🔍 Melhor conjunto de parâmetros: {'svr__C': 1, 'svr__epsilon': 0.2, 'svr__gamma': 'auto'}
✅ Pipeline final salvo: svr_A3_t3.joblib

⏳ Treinando SVR | A3 | t+5 | look_back=5
Fitting 5 folds for each of 18 candidates, totalling 90 fits
🔍 Melhor conjunto de parâmetros: {'svr__C': 1, 'svr__epsilon': 0.01, 'svr__gamma': 'auto'}
✅ Pipeline final salvo: svr_A3_t5.joblib

⏳ Treinando SVR | A3 | t+7 | look_back=10
Fitting 5 folds for each of 18 candidates, totalling 90 fits
🔍 Melhor conjunto de parâmetros: {'svr__C': 1, 'svr__epsilon': 0.1, 'svr__gamma': 'auto'}
✅ Pipeline final salvo: svr_A3_t7.joblib

⏳ 

## Abordagem alternativa

In [5]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import datetime
import logging
from scipy import stats

# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("wavelet_svr_training.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

def criar_dataset_multi_step(series, look_back=10, passo=1, feature_engineering=True):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        features = series[i:i+look_back].copy()
        if feature_engineering:
            tendencia = features[-1] - features[0]
            media_movel = np.mean(features)
            volatilidade = np.std(features)
            features = np.append(features, [tendencia, media_movel, volatilidade])
        X.append(features)
        y.append(series[i+look_back+passo-1])
    return np.array(X), np.array(y)

def avaliar_modelo(modelo, X_test, y_test, nome_modelo, coluna, passo):
    y_pred = modelo.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / np.maximum(np.abs(y_test), 1e-10))) * 100

    os.makedirs("resultados", exist_ok=True)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
    plt.xlabel('Valores Reais')
    plt.ylabel('Valores Previstos')
    plt.title(f'Real vs Previsto - {coluna} (t+{passo})')
    plt.subplot(1, 2, 2)
    n_show = min(100, len(y_test))
    plt.plot(y_test[:n_show], label='Real', alpha=0.7)
    plt.plot(y_pred[:n_show], label='Previsto', alpha=0.7)
    plt.legend()
    plt.title(f'Previsão vs Real - {coluna} (t+{passo})')
    plt.tight_layout()
    plt.savefig(f"resultados/{coluna}_t{passo}_resultados.png")
    plt.close()

    return {"MAE": mae, "RMSE": rmse, "R2": r2, "MAPE": mape}

def salvar_resultados_parciais(resultados, coluna):
    os.makedirs("resultados", exist_ok=True)
    for passo, modelos in resultados.items():
        resultados_passo_df = pd.DataFrame([
            {
                'Componente': coluna,
                'Horizonte': passo,
                'Modelo': modelo,
                'Look_Back': info['look_back'],
                'Feature_Engineering': info['feature_engineering'],
                'MAE': info['metricas']['MAE'],
                'RMSE': info['metricas']['RMSE'],
                'R2': info['metricas']['R2'],
                'MAPE': info['metricas']['MAPE']
            }
            for modelo, info in modelos.items()
        ])
        resultados_passo_df.to_csv(f"resultados/{coluna}_t{passo}_resultados.csv", index=False)

def treinar_svr_multiplos_passos(caminho_csv, coluna, passos=[1, 3, 7, 30], testar_diferentes_scalers=True, feature_engineering=True):
    logger.info(f"Iniciando treinamento para componente: {coluna}")
    os.makedirs("modelos", exist_ok=True)
    try:
        dados_df = pd.read_csv(caminho_csv)
        dados = dados_df[coluna].values.reshape(-1)
    except:
        dados = dados_df.iloc[:, 0].values.reshape(-1)
    dados = pd.Series(dados).interpolate().ffill().bfill().values
    resultados = {passo: {} for passo in passos}
    scalers = [('RobustScaler', RobustScaler()), ('StandardScaler', StandardScaler())] if testar_diferentes_scalers else [('RobustScaler', RobustScaler())]

    for passo in passos:
        if passo in [1, 3, 7]:
            look_back_values = [5]
        elif passo == 30:
            look_back_values = [10]
        else:
            logger.warning(f"Passo {passo} não suportado explicitamente. Usando look_back=10 por padrão.")
            look_back_values = [10]

        for look_back in look_back_values:
            X, y = criar_dataset_multi_step(dados, look_back, passo, feature_engineering)
            split = int(len(X) * 0.8)
            X_train, X_test = X[:split], X[split:]
            y_train, y_test = y[:split], y[split:]

            for scaler_nome, scaler in scalers:
                pipeline = Pipeline([('scaler', scaler), ('svr', SVR())])
                param_grid = {
                    'svr__C': [1, 10],
                    'svr__epsilon': [0.01, 0.1],
                    'svr__gamma': ['scale', 0.1],
                    'svr__kernel': ['rbf']
                }
                tscv = TimeSeriesSplit(n_splits=5)
                grid = GridSearchCV(pipeline, param_grid, scoring='neg_mean_absolute_error', cv=tscv, n_jobs=-1, verbose=0)
                try:
                    grid.fit(X_train, y_train)
                    best = grid.best_estimator_
                    metricas = avaliar_modelo(best, X_test, y_test, f"SVR_{scaler_nome}_lb{look_back}", coluna, passo)
                    resultados[passo][f"SVR_{scaler_nome}_lb{look_back}"] = {
                        'metricas': metricas,
                        'parametros': grid.best_params_,
                        'look_back': look_back,
                        'feature_engineering': feature_engineering
                    }
                    joblib.dump(best, f"modelos/svr_{coluna}_t{passo}_{scaler_nome}_lb{look_back}.joblib")
                except Exception as e:
                    logger.error(f"Erro no GridSearchCV: {e}")

    salvar_resultados_parciais(resultados, coluna)
    logger.info(f"Modelos e resultados salvos para componente {coluna}")
    return resultados

def treinar_componentes_wavelet(caminho_csv, componentes=['d1', 'd2', 'd3'], passos=[1, 3, 7, 30]):
    for componente in componentes:
        treinar_svr_multiplos_passos(caminho_csv, componente, passos)

# Exemplo de execução:
# treinar_componentes_wavelet("dados_wavelet.csv", componentes=['d1'], passos=[1, 3, 5])


In [6]:
# Exemplo de uso
if __name__ == "__main__":
    # Treinar modelos individualmente para cada componente conforme especificado
    logger.info("Iniciando treinamento do componente D3...")
    treinar_svr_multiplos_passos("D3_component.csv", "D3")
    
    logger.info("Iniciando treinamento do componente D2...")
    treinar_svr_multiplos_passos("D2_component.csv", "D2")
    
    logger.info("Iniciando treinamento do componente D1...")
    treinar_svr_multiplos_passos("D1_component.csv", "D1")

2025-05-17 22:05:53,770 - INFO - Iniciando treinamento do componente D3...
2025-05-17 22:05:53,771 - INFO - Iniciando treinamento para componente: D3
2025-05-17 22:07:57,513 - INFO - Modelos e resultados salvos para componente D3
2025-05-17 22:07:57,513 - INFO - Iniciando treinamento do componente D2...
2025-05-17 22:07:57,514 - INFO - Iniciando treinamento para componente: D2
2025-05-17 22:10:07,579 - INFO - Modelos e resultados salvos para componente D2
2025-05-17 22:10:07,581 - INFO - Iniciando treinamento do componente D1...
2025-05-17 22:10:07,583 - INFO - Iniciando treinamento para componente: D1
2025-05-17 22:12:24,267 - INFO - Modelos e resultados salvos para componente D1


## Salvando Random Forest

In [8]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import os
import joblib

def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i + look_back])
        y.append(series[i + look_back + passo - 1])
    return np.array(X), np.array(y)

def treinar_rf_para_componentes(caminhos_componentes, passos=[1, 5, 7, 30]):
    os.makedirs("modelosRF", exist_ok=True)
    os.makedirs("scalersRF", exist_ok=True)

    for comp, caminho_csv in caminhos_componentes.items():
        df = pd.read_csv(caminho_csv)
        serie = df[comp.upper()].values.reshape(-1, 1)
        serie = pd.Series(serie.flatten()).interpolate().ffill().bfill().values.reshape(-1, 1)

        for passo in passos:
            look_back = 5 if passo in [1, 5, 7] else 10

            print(f"🌲 Treinando RandomForest para {comp.upper()} | t+{passo} (look_back={look_back})")

            # Normalizar
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
            serie_scaled = scaler.fit_transform(serie)

            joblib.dump(scaler, f"scalersRF/scaler_{comp}_t{passo}.joblib")

            # Criar dataset
            X, y = criar_dataset_multi_step(serie_scaled, look_back=look_back, passo=passo)
            split = int(0.8 * len(X))
            X_train, y_train = X[:split], y[:split]
            X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]))

            # Modelo mais robusto
            model = RandomForestRegressor(
                n_estimators=200,
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42,
                n_jobs=-1
            )
            model.fit(X_train, y_train.ravel())

            # Salvar modelo
            joblib.dump(model, f"modelosRF/rf_{comp}_t{passo}.joblib")
            print(f"✅ Modelo salvo: rf_{comp}_t{passo}.joblib")


In [9]:
treinar_rf_para_componentes({
    'd1': 'D1_component.csv',
    'd2': 'D2_component.csv',
    'd3': 'D3_component.csv'
})


🌲 Treinando RandomForest para D1 | t+1 (look_back=5)
✅ Modelo salvo: rf_d1_t1.joblib
🌲 Treinando RandomForest para D1 | t+5 (look_back=5)
✅ Modelo salvo: rf_d1_t5.joblib
🌲 Treinando RandomForest para D1 | t+7 (look_back=5)
✅ Modelo salvo: rf_d1_t7.joblib
🌲 Treinando RandomForest para D1 | t+30 (look_back=10)
✅ Modelo salvo: rf_d1_t30.joblib
🌲 Treinando RandomForest para D2 | t+1 (look_back=5)
✅ Modelo salvo: rf_d2_t1.joblib
🌲 Treinando RandomForest para D2 | t+5 (look_back=5)
✅ Modelo salvo: rf_d2_t5.joblib
🌲 Treinando RandomForest para D2 | t+7 (look_back=5)
✅ Modelo salvo: rf_d2_t7.joblib
🌲 Treinando RandomForest para D2 | t+30 (look_back=10)
✅ Modelo salvo: rf_d2_t30.joblib
🌲 Treinando RandomForest para D3 | t+1 (look_back=5)
✅ Modelo salvo: rf_d3_t1.joblib
🌲 Treinando RandomForest para D3 | t+5 (look_back=5)
✅ Modelo salvo: rf_d3_t5.joblib
🌲 Treinando RandomForest para D3 | t+7 (look_back=5)
✅ Modelo salvo: rf_d3_t7.joblib
🌲 Treinando RandomForest para D3 | t+30 (look_back=10)
✅ Mo

## Random forest com mais contexto

In [3]:
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler
import pandas as pd
import numpy as np
import joblib
import json
import time

def criar_dataset_multi_step(series, look_back=10, passo=1, adicionar_features=True):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        # Extrair a sequência base
        sequencia = series[i:i + look_back].flatten()
        
        if adicionar_features:
            # Adicionar características para detecção de picos
            media_movel = np.mean(sequencia)
            std_movel = np.std(sequencia)
            max_local = np.max(sequencia)
            min_local = np.min(sequencia)
            amplitude = max_local - min_local
            tendencia = sequencia[-1] - sequencia[0]
            
            # Derivada (taxa de mudança)
            derivada = np.diff(sequencia)
            derivada_mean = np.mean(derivada) if len(derivada) > 0 else 0
            derivada_std = np.std(derivada) if len(derivada) > 0 else 0
            
            # Segunda derivada (aceleração)
            segunda_derivada = np.diff(derivada) if len(derivada) > 1 else np.array([0])
            segunda_derivada_mean = np.mean(segunda_derivada) if len(segunda_derivada) > 0 else 0
            
            # Features adicionais
            features = np.array([media_movel, std_movel, max_local, min_local, 
                               amplitude, tendencia, derivada_mean, derivada_std, 
                               segunda_derivada_mean])
            
            # Combinar features originais com as novas features
            X.append(np.concatenate([sequencia, features]))
        else:
            X.append(sequencia)
            
        y.append(series[i + look_back + passo - 1])
    
    return np.array(X), np.array(y)

def treinar_rf_para_componentes(caminhos_componentes, passos=[1], rf_params=None, look_back=None):
    """
    Treina modelos RandomForest para componentes wavelet.
    
    Args:
        caminhos_componentes: Dicionário com nome do componente e caminho do arquivo CSV
        passos: Lista de horizontes de previsão (t+n)
        rf_params: Dicionário com parâmetros para o RandomForestRegressor
        look_back: Valor de look_back para a janela de histórico. Se None, será calculado com base no passo.
    
    Returns:
        Dicionário com metadados dos modelos treinados
    """
    # Tempo de início
    tempo_inicio = time.time()
    
    # Parâmetros padrão do RandomForest se não forem especificados
    if rf_params is None:
        rf_params = {
            'n_estimators': 200,
            'max_depth': 15,
            'min_samples_split': 5,
            'min_samples_leaf': 1,
            'max_features': 'sqrt',
            'bootstrap': True,
            'random_state': 42,
            'n_jobs': -1
        }
    
    # Criar diretórios necessários
    os.makedirs("modelosRF", exist_ok=True)
    os.makedirs("scalersRF", exist_ok=True)
    os.makedirs("configsRF", exist_ok=True)
    
    # Dicionário para armazenar metadados de todos os modelos
    metadados_modelos = {}
    
    # Exibir configuração
    print("\n⚙️ Configuração de Treinamento:")
    print(f"   - Componentes: {list(caminhos_componentes.keys())}")
    print(f"   - Horizontes de previsão: {passos}")
    print(f"   - Parâmetros RF:")
    for param, valor in rf_params.items():
        print(f"      - {param}: {valor}")
    
    # Percorrer cada componente
    for comp, caminho_csv in caminhos_componentes.items():
        tempo_componente = time.time()
        print(f"\n🔍 Processando componente: {comp.upper()}")
        
        try:
            # Carregar dados
            df = pd.read_csv(caminho_csv)
            
            # Garantir que o componente exista no DataFrame
            coluna_componente = comp.upper()
            if coluna_componente not in df.columns:
                print(f"⚠️ Coluna {coluna_componente} não encontrada em {caminho_csv}")
                continue
                
            serie = df[coluna_componente].values.reshape(-1, 1)
            
            # Limpeza e interpolação de dados faltantes
            serie = pd.Series(serie.flatten()).interpolate().ffill().bfill().values.reshape(-1, 1)
            
            # Verificar se há valores infinitos ou NaN
            if np.isnan(serie).any() or np.isinf(serie).any():
                print(f"⚠️ Valores NaN ou infinitos encontrados em {comp}. Aplicando limpeza...")
                serie = np.nan_to_num(serie, nan=np.nanmean(serie), posinf=np.nanmax(serie), neginf=np.nanmin(serie))
            
            # Tamanho do dataset
            print(f"   - Tamanho do dataset: {len(serie)} pontos")
            
            # Metadados específicos do componente
            metadados_modelos[comp] = {}
            
            # Treinar para cada horizonte de previsão
            for passo in passos:
                tempo_passo = time.time()
                
                # Calcular look_back se não foi especificado
                lookback_atual = look_back if look_back is not None else max(10, passo // 2)
                
                print(f"\n🌲 Treinando RandomForest para {comp.upper()} | t+{passo} (look_back={lookback_atual})")
                
                # Usar RobustScaler para lidar melhor com outliers
                scaler = RobustScaler()
                serie_scaled = scaler.fit_transform(serie)
                
                # Salvar scaler
                scaler_filename = f"scalersRF/scaler_{comp}_t{passo}.joblib"
                joblib.dump(scaler, scaler_filename)
                
                # Criar dataset com features adicionais
                X, y = criar_dataset_multi_step(serie_scaled, look_back=lookback_atual, passo=passo, adicionar_features=True)
                
                # Informações sobre o dataset
                print(f"   - Dataset: {X.shape[0]} amostras, {X.shape[1]} features")
                
                # Divisão treino/teste
                split = int(0.8 * len(X))
                X_train = X[:split]
                y_train = y[:split]
                
                # Dicionário para armazenar metadados do modelo
                modelo_info = {
                    'componente': comp,
                    'passo': passo,
                    'look_back': lookback_atual,
                    'num_features': X.shape[1],
                    'num_amostras_treino': len(X_train),
                    'data_treinamento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'usa_features_adicionais': True,
                    'parametros': rf_params
                }
                
                # Criar e treinar modelo com os parâmetros fornecidos
                print(f"   - Iniciando treinamento...")
                model = RandomForestRegressor(**rf_params)
                model.fit(X_train, y_train.ravel())
                
                # Se OOB score estiver disponível, salvar
                if hasattr(model, 'oob_score_'):
                    modelo_info['oob_score'] = float(model.oob_score_)
                    print(f"   - OOB Score: {model.oob_score_:.4f}")
                
                # Calcular e mostrar importância das features
                if len(model.feature_importances_) == X.shape[1]:
                    # Número base de features originais
                    num_features_orig = lookback_atual
                    
                    # Importância das features originais (valores passados)
                    imp_features_orig = model.feature_importances_[:num_features_orig].sum()
                    
                    # Importância das features engineered
                    imp_features_eng = model.feature_importances_[num_features_orig:].sum()
                    
                    print(f"   - Importância das features:")
                    print(f"      - Features originais: {imp_features_orig:.4f} ({imp_features_orig*100:.1f}%)")
                    print(f"      - Features engineered: {imp_features_eng:.4f} ({imp_features_eng*100:.1f}%)")
                    
                    # Top 5 features mais importantes
                    top_indices = np.argsort(model.feature_importances_)[-5:][::-1]
                    
                    # Nomes das features
                    feature_names = [f"t-{i+1}" for i in range(lookback_atual)]
                    feature_names.extend(["média", "std", "max", "min", "amplitude", 
                                          "tendência", "derivada_média", "derivada_std", 
                                          "2ª_derivada_média"])
                    
                    print(f"      - Top 5 features:")
                    for i, idx in enumerate(top_indices):
                        if idx < len(feature_names):
                            print(f"         {i+1}. {feature_names[idx]}: {model.feature_importances_[idx]:.4f}")
                        else:
                            print(f"         {i+1}. Feature {idx}: {model.feature_importances_[idx]:.4f}")
                    
                    # Salvar importância das features no modelo_info
                    modelo_info['importancia_features'] = {
                        'features_originais': float(imp_features_orig),
                        'features_engineered': float(imp_features_eng),
                        'top_5_indices': top_indices.tolist(),
                        'top_5_valores': model.feature_importances_[top_indices].tolist(),
                        'todas_importancias': model.feature_importances_.tolist()
                    }
                
                # Salvar modelo
                modelo_filename = f"modelosRF/rf_{comp}_t{passo}.joblib"
                joblib.dump(model, modelo_filename)
                
                # Salvar metadados do modelo
                config_filename = f"configsRF/config_{comp}_t{passo}.json"
                with open(config_filename, 'w') as f:
                    json.dump(modelo_info, f, indent=4)
                
                # Adicionar informação ao dicionário global
                metadados_modelos[comp][f't{passo}'] = modelo_info
                
                # Tempo de treinamento para este horizonte
                tempo_passo_fim = time.time() - tempo_passo
                print(f"✅ Modelo salvo: {modelo_filename}")
                print(f"📝 Configuração salva: {config_filename}")
                print(f"⏱️ Tempo de treinamento para t+{passo}: {tempo_passo_fim:.2f} segundos")
        
        except Exception as e:
            print(f"❌ Erro ao processar componente {comp}: {str(e)}")
        
        # Tempo para este componente
        tempo_componente_fim = time.time() - tempo_componente
        print(f"⏱️ Tempo total para componente {comp.upper()}: {tempo_componente_fim:.2f} segundos")
    
    # Salvar metadados globais
    with open("configsRF/metadados_global.json", 'w') as f:
        json.dump(metadados_modelos, f, indent=4)
    
    print(f"\n📊 Metadados de todos os modelos salvos em: configsRF/metadados_global.json")
    
    # Tempo total
    tempo_total = time.time() - tempo_inicio
    minutos = int(tempo_total // 60)
    segundos = int(tempo_total % 60)
    print(f"⏱️ Tempo total de execução: {minutos} minutos e {segundos} segundos")
    
    return metadados_modelos

def carregar_config_modelo(componente, passo):
    """Carrega a configuração de um modelo específico"""
    try:
        with open(f"configsRF/config_{componente}_t{passo}.json", 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"❌ Configuração não encontrada para {componente}_t{passo}")
        return None

# --------------------------------
# CÓDIGO PRINCIPAL PARA EXECUÇÃO
# --------------------------------

if __name__ == "__main__":
    print("🚀 Treinamento de RandomForest para componentes Wavelet de detalhe (D1, D2, D3)")
    
    # Definir os caminhos dos arquivos CSV para cada componente de detalhe
    caminhos_componentes = {
        'd1': 'D1_component.csv',
        'd2': 'D2_component.csv',
        'd3': 'D3_component.csv'
    }
    
    # Menu para seleção de configuração
    print("\n📋 Escolha uma configuração:")
    print("1. Configuração básica (rápida)")
    print("2. Configuração para detecção de picos")
    print("3. Configuração personalizada")
    
    opcao = input("Opção (1, 2 ou 3): ")
    
    # Definir parâmetros com base na escolha
    if opcao == "1":
        # Configuração básica e rápida
        rf_params = {
            'n_estimators': 100,
            'max_depth': 10,
            'min_samples_split': 5,
            'min_samples_leaf': 2,
            'max_features': 'sqrt',
            'bootstrap': True,
            'random_state': 42,
            'n_jobs': -1
        }
        look_back = 10
        passos = [1]  # Apenas t+1 para teste rápido
        
    elif opcao == "2":
        # Configuração otimizada para detecção de picos
        rf_params = {
            'n_estimators': 300,
            'max_depth': 15,
            'min_samples_split': 5,
            'min_samples_leaf': 1,  # 1 para maior sensibilidade a outliers (picos)
            'max_features': None,   # Usar todas as features
            'bootstrap': True,
            'random_state': 42,
            'n_jobs': -1
        }
        look_back = 15
        passos = [1, 7]  # t+1 e t+7 como amostra
        
    else:  # Opção 3 ou qualquer outra entrada
        # Configuração personalizada
        print("\n⚙️ Configure os parâmetros do RandomForest:")
        
        # Número de árvores
        try:
            n_estimators = int(input("Número de árvores (recomendado: 100-500) [200]: ") or "200")
        except ValueError:
            n_estimators = 200
            print("Valor inválido. Usando 200 árvores.")
        
        # Profundidade máxima
        try:
            max_depth_input = input("Profundidade máxima (recomendado: 10-20, 'None' para ilimitado) [15]: ") or "15"
            max_depth = None if max_depth_input.lower() == 'none' else int(max_depth_input)
        except ValueError:
            max_depth = 15
            print("Valor inválido. Usando profundidade 15.")
        
        # min_samples_split
        try:
            min_samples_split = int(input("min_samples_split (recomendado: 2-10) [5]: ") or "5")
        except ValueError:
            min_samples_split = 5
            print("Valor inválido. Usando 5.")
        
        # min_samples_leaf
        try:
            min_samples_leaf = int(input("min_samples_leaf (1 para mais sensibilidade a picos) [1]: ") or "1")
        except ValueError:
            min_samples_leaf = 1
            print("Valor inválido. Usando 1.")
        
        # max_features
        max_features_input = input("max_features ('sqrt', 'log2', 'None' para todas) [sqrt]: ") or "sqrt"
        if max_features_input.lower() == 'none':
            max_features = None
        else:
            max_features = max_features_input if max_features_input in ['sqrt', 'log2'] else 'sqrt'
        
        # Look-back
        try:
            look_back_input = input("look_back (recomendado: 10-20, 'None' para automático) [None]: ") or "None"
            look_back = None if look_back_input.lower() == 'none' else int(look_back_input)
        except ValueError:
            look_back = None
            print("Valor inválido. Usando look_back automático.")
        
        # Horizontes de previsão
        passos_input = input("Horizontes de previsão (separados por vírgula, ex: 1,7,30) [1]: ") or "1"
        try:
            passos = [int(p.strip()) for p in passos_input.split(",")]
        except ValueError:
            passos = [1]
            print("Valor inválido. Usando apenas horizonte t+1.")
        
        # Montar parâmetros
        rf_params = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'max_features': max_features,
            'bootstrap': True,
            'random_state': 42,
            'n_jobs': -1
        }
    
    # Confirmar configuração
    print("\n⚙️ Configuração selecionada:")
    print(f"   - RandomForest: {rf_params}")
    print(f"   - look_back: {'Automático' if look_back is None else look_back}")
    print(f"   - Horizontes: {passos}")
    
    confirma = input("\nConfirma esta configuração? (s/n): ").lower()
    
    if confirma == 's':
        # Inicia o treinamento
        print("\n🏃 Iniciando treinamento...")
        metadados = treinar_rf_para_componentes(caminhos_componentes, passos, rf_params, look_back)
        
        # Exibir resumo
        print("\n✅ Treinamento concluído!")
        print("📊 Resumo dos modelos treinados:")
        
        for comp, modelos in metadados.items():
            print(f"\n📌 Componente: {comp.upper()}")
            for passo_key, info in modelos.items():
                print(f"   - Passo {info['passo']}: look_back={info['look_back']}, features={info['num_features']}")
                if 'oob_score' in info:
                    print(f"     OOB Score: {info['oob_score']:.4f}")
    else:
        print("Treinamento cancelado.")

🚀 Treinamento de RandomForest para componentes Wavelet de detalhe (D1, D2, D3)

📋 Escolha uma configuração:
1. Configuração básica (rápida)
2. Configuração para detecção de picos
3. Configuração personalizada

⚙️ Configuração selecionada:
   - RandomForest: {'n_estimators': 300, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': True, 'random_state': 42, 'n_jobs': -1}
   - look_back: 15
   - Horizontes: [1, 7]

🏃 Iniciando treinamento...

⚙️ Configuração de Treinamento:
   - Componentes: ['d1', 'd2', 'd3']
   - Horizontes de previsão: [1, 7]
   - Parâmetros RF:
      - n_estimators: 300
      - max_depth: 15
      - min_samples_split: 5
      - min_samples_leaf: 1
      - max_features: None
      - bootstrap: True
      - random_state: 42
      - n_jobs: -1

🔍 Processando componente: D1
   - Tamanho do dataset: 7878 pontos

🌲 Treinando RandomForest para D1 | t+1 (look_back=15)
   - Dataset: 7863 amostras, 24 features
   - Iniciando treina

### Salvado oXGBoost

In [22]:
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
import joblib

# Função para criar dataset multi-step
def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i+look_back])
        y.append(series[i+look_back+passo-1])
    return np.array(X), np.array(y)

# Função para treinar XGBoost para múltiplos passos
def treinar_xgb_multiplos_passos(caminho_csv, coluna, passos=[1, 3, 5, 7, 30]):
    dados = pd.read_csv(caminho_csv)[coluna].values
    dados = dados.reshape(-1)  # já normalizado externamente

    for passo in passos:
        look_back = 5 if passo in [1, 3, 5] else 10

        print(f"🚀 Treinando XGBoost para {coluna} | t+{passo} (look_back={look_back})")

        X, y = criar_dataset_multi_step(dados, look_back=look_back, passo=passo)

        split = int(0.7 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = XGBRegressor()
        model.fit(X_train, y_train)

        # Salvar modelo
        joblib.dump(model, f"xgb_{coluna}_t{passo}.joblib")
        print(f"✅ Modelo salvo: xgb_{coluna}_t{passo}.joblib")




In [ ]:
treinar_xgb_multiplos_passos("D3_component.csv", "D3")
treinar_xgb_multiplos_passos("D2_component.csv", "D2")
treinar_xgb_multiplos_passos("D1_component.csv", "D1")


🚀 Treinando XGBoost para D3 | t+1 (look_back=5)
✅ Modelo salvo: xgb_D3_t1.joblib
🚀 Treinando XGBoost para D3 | t+3 (look_back=5)
✅ Modelo salvo: xgb_D3_t3.joblib
🚀 Treinando XGBoost para D3 | t+5 (look_back=5)
✅ Modelo salvo: xgb_D3_t5.joblib
🚀 Treinando XGBoost para D3 | t+7 (look_back=10)
✅ Modelo salvo: xgb_D3_t7.joblib
🚀 Treinando XGBoost para D3 | t+30 (look_back=10)
✅ Modelo salvo: xgb_D3_t30.joblib
🚀 Treinando XGBoost para D2 | t+1 (look_back=5)
✅ Modelo salvo: xgb_D2_t1.joblib
🚀 Treinando XGBoost para D2 | t+3 (look_back=5)
✅ Modelo salvo: xgb_D2_t3.joblib
🚀 Treinando XGBoost para D2 | t+5 (look_back=5)
✅ Modelo salvo: xgb_D2_t5.joblib
🚀 Treinando XGBoost para D2 | t+7 (look_back=10)
✅ Modelo salvo: xgb_D2_t7.joblib
🚀 Treinando XGBoost para D2 | t+30 (look_back=10)
✅ Modelo salvo: xgb_D2_t30.joblib
🚀 Treinando XGBoost para D1 | t+1 (look_back=5)
✅ Modelo salvo: xgb_D1_t1.joblib
🚀 Treinando XGBoost para D1 | t+3 (look_back=5)
✅ Modelo salvo: xgb_D1_t3.joblib
🚀 Treinando XGBoost 

## Treinar com ExtraTreesRegressor

In [24]:
from sklearn.ensemble import ExtraTreesRegressor
import pandas as pd
import numpy as np
import joblib

# Função para criar dataset multi-step
def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i+look_back])
        y.append(series[i+look_back+passo-1])
    return np.array(X), np.array(y)

# Função para treinar ExtraTrees para múltiplos passos
def treinar_etr_multiplos_passos(caminho_csv, coluna, passos=[1, 3, 5, 7, 30]):
    dados = pd.read_csv(caminho_csv)[coluna].values
    dados = dados.reshape(-1)  # já normalizado externamente

    for passo in passos:
        look_back = 5 if passo in [1, 3, 5] else 10

        print(f"🌲 Treinando ExtraTrees para {coluna} | t+{passo} (look_back={look_back})")

        X, y = criar_dataset_multi_step(dados, look_back=look_back, passo=passo)

        split = int(0.7 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = ExtraTreesRegressor()
        model.fit(X_train, y_train)

        # Salvar modelo
        joblib.dump(model, f"etr_{coluna}_t{passo}.joblib")
        print(f"✅ Modelo salvo: etr_{coluna}_t{passo}.joblib")


In [25]:
treinar_etr_multiplos_passos("D3_component.csv", "D3")
treinar_etr_multiplos_passos("D2_component.csv", "D2")
treinar_etr_multiplos_passos("D1_component.csv", "D1")


🌲 Treinando ExtraTrees para D3 | t+1 (look_back=5)
✅ Modelo salvo: etr_D3_t1.joblib
🌲 Treinando ExtraTrees para D3 | t+3 (look_back=5)
✅ Modelo salvo: etr_D3_t3.joblib
🌲 Treinando ExtraTrees para D3 | t+5 (look_back=5)
✅ Modelo salvo: etr_D3_t5.joblib
🌲 Treinando ExtraTrees para D3 | t+7 (look_back=10)
✅ Modelo salvo: etr_D3_t7.joblib
🌲 Treinando ExtraTrees para D3 | t+30 (look_back=10)
✅ Modelo salvo: etr_D3_t30.joblib
🌲 Treinando ExtraTrees para D2 | t+1 (look_back=5)
✅ Modelo salvo: etr_D2_t1.joblib
🌲 Treinando ExtraTrees para D2 | t+3 (look_back=5)
✅ Modelo salvo: etr_D2_t3.joblib
🌲 Treinando ExtraTrees para D2 | t+5 (look_back=5)
✅ Modelo salvo: etr_D2_t5.joblib
🌲 Treinando ExtraTrees para D2 | t+7 (look_back=10)
✅ Modelo salvo: etr_D2_t7.joblib
🌲 Treinando ExtraTrees para D2 | t+30 (look_back=10)
✅ Modelo salvo: etr_D2_t30.joblib
🌲 Treinando ExtraTrees para D1 | t+1 (look_back=5)
✅ Modelo salvo: etr_D1_t1.joblib
🌲 Treinando ExtraTrees para D1 | t+3 (look_back=5)
✅ Modelo salvo: e

## Treinando para LigthGBM

In [29]:
from lightgbm import LGBMRegressor
import pandas as pd
import numpy as np
import joblib

def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i+look_back])
        y.append(series[i+look_back+passo-1])
    return np.array(X), np.array(y)

def treinar_lgbm_multiplos_passos(caminho_csv, coluna, passos=[1, 3, 5, 7, 30]):
    dados = pd.read_csv(caminho_csv)[coluna].values
    dados = dados.reshape(-1)

    for passo in passos:
        look_back = 5 if passo in [1, 3, 5] else 10

        print(f"💡 Treinando LightGBM para {coluna} | t+{passo} (look_back={look_back})")
        X, y = criar_dataset_multi_step(dados, look_back=look_back, passo=passo)

        split = int(0.7 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = LGBMRegressor(min_data_in_leaf=3,min_split_gain=0.0001,verbose=-1)
        model.fit(X_train, y_train)

        joblib.dump(model, f"lgbm_{coluna}_t{passo}.joblib")
        print(f"✅ Modelo salvo: lgbm_{coluna}_t{passo}.joblib")


In [30]:
treinar_lgbm_multiplos_passos("D1_component.csv", "D1")
treinar_lgbm_multiplos_passos("D2_component.csv", "D2")
treinar_lgbm_multiplos_passos("D3_component.csv", "D3")

💡 Treinando LightGBM para D1 | t+1 (look_back=5)
✅ Modelo salvo: lgbm_D1_t1.joblib
💡 Treinando LightGBM para D1 | t+3 (look_back=5)
✅ Modelo salvo: lgbm_D1_t3.joblib
💡 Treinando LightGBM para D1 | t+5 (look_back=5)
✅ Modelo salvo: lgbm_D1_t5.joblib
💡 Treinando LightGBM para D1 | t+7 (look_back=10)
✅ Modelo salvo: lgbm_D1_t7.joblib
💡 Treinando LightGBM para D1 | t+30 (look_back=10)
✅ Modelo salvo: lgbm_D1_t30.joblib
💡 Treinando LightGBM para D2 | t+1 (look_back=5)
✅ Modelo salvo: lgbm_D2_t1.joblib
💡 Treinando LightGBM para D2 | t+3 (look_back=5)
✅ Modelo salvo: lgbm_D2_t3.joblib
💡 Treinando LightGBM para D2 | t+5 (look_back=5)
✅ Modelo salvo: lgbm_D2_t5.joblib
💡 Treinando LightGBM para D2 | t+7 (look_back=10)
✅ Modelo salvo: lgbm_D2_t7.joblib
💡 Treinando LightGBM para D2 | t+30 (look_back=10)
✅ Modelo salvo: lgbm_D2_t30.joblib
💡 Treinando LightGBM para D3 | t+1 (look_back=5)
✅ Modelo salvo: lgbm_D3_t1.joblib
💡 Treinando LightGBM para D3 | t+3 (look_back=5)
✅ Modelo salvo: lgbm_D3_t3.job

## Treinando com CatBoosting

In [32]:
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
import joblib

def treinar_catboost_multiplos_passos(caminho_csv, coluna, passos=[1, 3, 5, 7, 30]):
    dados = pd.read_csv(caminho_csv)[coluna].values
    dados = dados.reshape(-1)

    for passo in passos:
        look_back = 5 if passo in [1, 3, 5] else 10

        print(f"🐱 Treinando CatBoost para {coluna} | t+{passo} (look_back={look_back})")
        X, y = criar_dataset_multi_step(dados, look_back=look_back, passo=passo)

        split = int(0.7 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = CatBoostRegressor(verbose=0)
        model.fit(X_train, y_train)

        joblib.dump(model, f"catboost_{coluna}_t{passo}.joblib")
        print(f"✅ Modelo salvo: catboost_{coluna}_t{passo}.joblib")


In [33]:
treinar_catboost_multiplos_passos("D1_component.csv", "D1")
treinar_catboost_multiplos_passos("D2_component.csv", "D2")
treinar_catboost_multiplos_passos("D3_component.csv", "D3")

🐱 Treinando CatBoost para D1 | t+1 (look_back=5)
✅ Modelo salvo: catboost_D1_t1.joblib
🐱 Treinando CatBoost para D1 | t+3 (look_back=5)
✅ Modelo salvo: catboost_D1_t3.joblib
🐱 Treinando CatBoost para D1 | t+5 (look_back=5)
✅ Modelo salvo: catboost_D1_t5.joblib
🐱 Treinando CatBoost para D1 | t+7 (look_back=10)
✅ Modelo salvo: catboost_D1_t7.joblib
🐱 Treinando CatBoost para D1 | t+30 (look_back=10)
✅ Modelo salvo: catboost_D1_t30.joblib
🐱 Treinando CatBoost para D2 | t+1 (look_back=5)
✅ Modelo salvo: catboost_D2_t1.joblib
🐱 Treinando CatBoost para D2 | t+3 (look_back=5)
✅ Modelo salvo: catboost_D2_t3.joblib
🐱 Treinando CatBoost para D2 | t+5 (look_back=5)
✅ Modelo salvo: catboost_D2_t5.joblib
🐱 Treinando CatBoost para D2 | t+7 (look_back=10)
✅ Modelo salvo: catboost_D2_t7.joblib
🐱 Treinando CatBoost para D2 | t+30 (look_back=10)
✅ Modelo salvo: catboost_D2_t30.joblib
🐱 Treinando CatBoost para D3 | t+1 (look_back=5)
✅ Modelo salvo: catboost_D3_t1.joblib
🐱 Treinando CatBoost para D3 | t+3 

## Usar uma MLP

In [36]:
from sklearn.neural_network import MLPRegressor
import pandas as pd
import numpy as np
import joblib

# Função para criar dataset com saída média dos próximos `passo` valores
def criar_dataset_com_media_futura(series, look_back=5, passo=3):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        entrada = series[i:i+look_back]
        janela_futura = series[i+look_back:i+look_back+passo]
        media_futura = np.mean(janela_futura)
        X.append(entrada)
        y.append(media_futura)
    return np.array(X), np.array(y)

# Função para treinar MLP com saída média futura
def treinar_mlp_media_futura(caminho_csv, coluna, passos=[1, 3, 5, 7, 30]):
    dados = pd.read_csv(caminho_csv)[coluna].values
    dados = dados.reshape(-1)

    for passo in passos:
        look_back = 5 if passo in [1, 3, 5] else 10
        print(f"🔁 Treinando MLP para média futura de {coluna} | passo={passo} (look_back={look_back})")

        X, y = criar_dataset_com_media_futura(dados, look_back=look_back, passo=passo)

        split = int(0.7 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = MLPRegressor(
                                hidden_layer_sizes=(150, 100, 50),
                                activation='relu',
                                solver='adam',
                                learning_rate='adaptive',
                                max_iter=1000,
                                early_stopping=True,
                                validation_fraction=0.1,
                                n_iter_no_change=20
                        )

        model.fit(X_train, y_train)

        joblib.dump(model, f"mlpavg_{coluna}_t{passo}.joblib")
        print(f"✅ Modelo salvo: mlpavg_{coluna}_t{passo}.joblib")


In [37]:
treinar_mlp_media_futura("D1_component.csv", "D1")
treinar_mlp_media_futura("D2_component.csv", "D2")
treinar_mlp_media_futura("D3_component.csv", "D3")


🔁 Treinando MLP para média futura de D1 | passo=1 (look_back=5)
✅ Modelo salvo: mlpavg_D1_t1.joblib
🔁 Treinando MLP para média futura de D1 | passo=3 (look_back=5)
✅ Modelo salvo: mlpavg_D1_t3.joblib
🔁 Treinando MLP para média futura de D1 | passo=5 (look_back=5)
✅ Modelo salvo: mlpavg_D1_t5.joblib
🔁 Treinando MLP para média futura de D1 | passo=7 (look_back=10)
✅ Modelo salvo: mlpavg_D1_t7.joblib
🔁 Treinando MLP para média futura de D1 | passo=30 (look_back=10)
✅ Modelo salvo: mlpavg_D1_t30.joblib
🔁 Treinando MLP para média futura de D2 | passo=1 (look_back=5)
✅ Modelo salvo: mlpavg_D2_t1.joblib
🔁 Treinando MLP para média futura de D2 | passo=3 (look_back=5)
✅ Modelo salvo: mlpavg_D2_t3.joblib
🔁 Treinando MLP para média futura de D2 | passo=5 (look_back=5)
✅ Modelo salvo: mlpavg_D2_t5.joblib
🔁 Treinando MLP para média futura de D2 | passo=7 (look_back=10)
✅ Modelo salvo: mlpavg_D2_t7.joblib
🔁 Treinando MLP para média futura de D2 | passo=30 (look_back=10)
✅ Modelo salvo: mlpavg_D2_t30

## Criando uam MLP refinada

In [ ]:
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd
import pywt
import joblib

def criar_dataset_media(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        entrada = series[i:i+look_back]
        janela_futura = series[i+look_back:i+look_back+passo]
        media_futura = np.mean(janela_futura)
        X.append(entrada)
        y.append(media_futura)
    return np.array(X), np.array(y)

def treinar_mlp_para_a3(path_csv_norm, passos=[1, 3, 5, 7, 30]):
    df = pd.read_csv(path_csv_norm)
    serie_norm = df["RMDM_Norm"].values
    coeffs = pywt.wavedec(serie_norm, 'db4', level=3)
    a3 = pywt.upcoef('a', coeffs[0], 'db4', level=3)[:len(serie_norm)]

    for passo in passos:
        look_back = 5 if passo in [1, 3, 5] else 10
        print(f"🔁 Treinando MLP para A3 | t+{passo} (look_back={look_back})")

        X, y = criar_dataset_media(a3, look_back=look_back, passo=passo)

        # Remoção de NaNs (precaução)
        mask = ~np.isnan(X).any(axis=1) & ~np.isnan(y)
        X, y = X[mask], y[mask]

        model = MLPRegressor(
            hidden_layer_sizes=(200, 150, 100, 50),  # 🔼 Mais capacidade
            activation='relu',
            solver='adam',
            alpha=0.0005, # 🔧 Regularização L2
            learning_rate='adaptive',
            learning_rate_init=0.001,
            early_stopping=True,
            validation_fraction=0.1,
            n_iter_no_change=30,
            max_iter=2000,
            random_state=42)
        model.fit(X, y)
        joblib.dump(model, f"mlp_a3_t{passo}.joblib")
        print(f"✅ Modelo salvo: mlp_a3_t{passo}.joblib")


In [47]:
treinar_mlp_para_a3("RMDM_litoral_norte_com_normalizado.csv")


🔁 Treinando MLP para A3 | t+1 (look_back=5)
✅ Modelo salvo: mlp_a3_t1.joblib
🔁 Treinando MLP para A3 | t+3 (look_back=5)
✅ Modelo salvo: mlp_a3_t3.joblib
🔁 Treinando MLP para A3 | t+5 (look_back=5)
✅ Modelo salvo: mlp_a3_t5.joblib
🔁 Treinando MLP para A3 | t+7 (look_back=10)
✅ Modelo salvo: mlp_a3_t7.joblib
🔁 Treinando MLP para A3 | t+30 (look_back=10)
✅ Modelo salvo: mlp_a3_t30.joblib
